In [68]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using Revise
using LinearAlgebra
using HybridRobotDynamics
using SiLQR

@show HybridRobotDynamics.five_link_walker


  Activating project at `~/Workspace/16745/project/SiLQR.jl`


UndefVarError: UndefVarError: `five_link_walker` not defined

In [39]:
"""
Solver Setup
"""

# Define RABBIT (Five-Link Walker) model parameters
g = 9.81  # gravity
ls = [0.5, 0.5, 0.5, 0.5, 0.5] # Link lengths (meters)
ms = [5.0, 5.0, 5.0, 5.0, 5.0] # Link masses (kg) 
Is = [0.1, 0.1, 0.1, 0.1, 0.1] # Link inertias (kg*m^2)
system = HybridRobotDynamics.five_link_walker(Is, ls, ms, g)

# Cost Matrices
Q = Diagonal(fill(10.0, 10))      # State cost
Qf = Diagonal(fill(100.0, 10))    # Final state cost
R = Diagonal(fill(1.0, 2))        # Input cost
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 40
Δt = 0.05
params = SiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory (walk in place)
xref = zeros(system.nx)
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [1e-2 * randn(system.nu) for k = 1:N-1] # start with some noise

# Initial state (small perturbation)
q0 = [0.1, -0.2, 0.2, -0.1, 0.0]  # angles
v0 = zeros(5)                    # velocities
params.x0 = vcat(q0, v0)

# Set initial mode
params.mI = :stance

nothing

UndefVarError: UndefVarError: `five_link_walker` not defined

In [ ]:
"""
Solve using SiLQR
"""

# Solve
sol = SiLQR.Solution(params)
cache = SiLQR.Cache(params)
@time SiLQR.solve!(sol, cache, params)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1, 3), xs; xlim=(-0.5, 0.5), ylim=(-0.5, 0.5))

nothing